## ACA Hosted MCP Server with Azure AI Foundry Agent

![MCP with AI Agent](./images/mcp_apim_with_ai_agent.png)

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4

### Setting Up the Environment 

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Creating our MCP Tool Object

In [ ]:
from agent_framework import HostedMCPTool

udemy_demo_mcp_tool = HostedMCPTool(
    name = "Udemy Demo MCP Tool",
    url = "https://kuljot-apim-standard.azure-api.net/star-wars-mcp-server/mcp" # URL needs to be populated when deployed on ACA (add a /mcp endpoint)
)

### Creating our MCP Agent

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Cell 2: Define async workflow
async def create_mcp_agent():
    # Create async Azure credential
    credential = AzureCliCredential()
    
    # Initialize the Azure AI Agent Client
    chat_client = AzureAIAgentClient(async_credential=credential,
                                     project_endpoint=project_endpoint,
                                     model_deployment_name=model)

    try:

        # Create the MCP Agent
        agent = chat_client.create_agent(
            name="MCP-Agent",
            instructions="You are a helpful AI Assistant",
            tools=udemy_demo_mcp_tool
        )

        print("Agent created successfully!")
        return agent

    finally:
        # Clean up async clients
        await chat_client.close()
        await credential.close()


In [ ]:
# Cell 3: Run the async function
agent = await create_mcp_agent()


### Creating an Async Function to Handle User Approvals and Interactions

In [ ]:
from agent_framework import ChatMessage

async def handle_approvals_with_thread(query: str, agent, thread):
    result = await agent.run(query, thread=thread, store=True)
    while len(result.user_input_requests) > 0:
        new_input = []
        for user_input_needed in result.user_input_requests:
            print(
                f"\n⚙️ Function call requested: {user_input_needed.function_call.name}"
                f"\nArguments: {user_input_needed.function_call.arguments}"
            )
            user_approval = input("Approve function call? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    return result


### Invoking our Agent

#### Some Queries to Try Out

##### For the Self Created MCP Server
- "Tell me something about the recipe with ID 12"
- "What are the ingredients for the recipe with ID 5?"
- "Tell me about the course with ID 3 for author Kuljot Singh Bakshi on Udemy."
- "What is the duration of the course with ID 7 on Udemy?"

##### For the Star Wars MCP Server
- "Tell me something about the star wars character with ID 4"
- "What is the height of the star wars character with ID 6?"

In [ ]:
thread = agent.get_new_thread()
query = "tell me about courses with rating greater than 4....which one has the maximum lectures?"
response = await handle_approvals_with_thread(query, agent, thread)

print("\n🧠 Assistant:\n", response)
